Отчет по тестовому заданию я решил написать в формате ноутбука с комментариями и рассуждениями в процессе работы.

Задача состоит в том, чтобы написать веб-сервис на основе обученной ML модели.

В качестве данные для работы есть: упакованная мадель micromodel.cbm, скрипт для получения предсказаний 
этой модели pick_regno.py и небольшой набор данных размером в 100 строк на которых можно проверить роботоспособность модели.

Для начала продумаем архитектуру микросервиса, для этого нужно понимать какую задачу он решает.
Очевидно что это классификатор на основе кэтбуста, информации о том какие классы он предсказывает у меня нет, но по таблице с фичами можно увидеть следующее:

In [12]:
import pandas as pd
import numpy as np
data = pd.read_csv('REST_ml_app/test_data.csv')
data.head(5)

,regno_recognize,afts_regno_ai,recognition_accuracy,afts_regno_ai_score,afts_regno_ai_char_scores,afts_regno_ai_length_scores,camera_type,camera_class,time_check,direction
0,А939НО196,А939НО190,6.400000,0.868917,"[0.9998925924301147, 0.9999872446060181, 0.999...","[3.2404470773350624e-10, 3.236617363011618e-10...",Стационарная,Астра-Трафик,2021-08-01 09:02:59,0
1,В154АУ797,В154АУ777,8.099999,0.651144,"[0.9999932050704956, 0.9999973773956299, 0.999...","[3.430950662464127e-11, 3.460254999199108e-11,...",Стационарная,Астра-Трафик,2021-08-01 11:24:40,1
2,О673ТВ198,О673ТВ190,99.000000,0.605001,"[0.9999418258666992, 0.9999479055404663, 0.999...","[1.1617498074656396e-09, 1.1486793738413326e-0...",Стационарная,Стрелка-Плюс,2021-08-01 11:50:28,1
3,А847ЕК198,А847ЕК190,14.250000,0.670378,"[0.9999666213989258, 0.9999755620956421, 0.999...","[3.664501813105403e-09, 3.674545556719977e-09,...",Стационарная,Астра-Трафик,2021-08-01 12:53:14,0
4,Н225АР797,Н225АР777,99.000000,0.667756,"[0.9999465942382812, 0.9999498128890991, 0.999...","[5.063955921258412e-09, 5.053997664816734e-09,...",Стационарная,Стрелка-Плюс,2021-08-01 13:48:23,0


Это результаты работы некой стационарной системы видеорегистрации проезжей части

Далее идут спекуляции: 

regno_recognize - скорее всего описывает как внутренний софт камеры определил номер автомобиля

afts_regno_ai - показывает как некая модель с нейронкой определяет номер автомобиля

recognition_accuracy, afts_regno_ai_score - метрики качества определителя и нейронки

afts_regno_ai_char_scores - вероятности предсказанных символов номера нейронкой

afts_regno_ai_length_scores	- не совсем понятно

camera_type, camera_class - характеристики камеры

direction - направление движения

Итак, вероятно имеется следующая структура: Есть камеры видеорегистрации на проезжей части (это точно не охраняемый объект поскольку в данных проблематично увидеть заезжающий и выезжающий автомобиль), некая система, которая стоит за рамками поставленной задачи детектирует автомобили в видеопотоке и проверяет их скорость/соблюдение полосы и тд.
В случае если системой детекции было обнаружено несоблюдение пдд (Если бы система фиксировала каждую машину а не условно нарушавшую пдд то данные были бы не так разрежены по временному ряду) данные о машине и изображение передаются дальше и определяется её номер, причем либо номер определяется двумя способами независимо: нейросетью и неким ПО камеры, либо нейросеть занимается улучшением качества определения номеров. После этого номера и прочие данные попадают ко мне в предоставленную модель, которая может например принимать решение правильно определен номер автомобиля или нет.

Глабольно мы имеем систему которая в режиме реального времени принимает решения на основе трансляции видео.

Таким образом, в сервис будут постоянно приходить новые данные в виде массива в котором содержится информация с фичами для модели которые нужно сразу обрабатывать

На этом спекулияции закончены, приступим к делу.

Исходя из этих рассуждений нужно сделать масштабируемый сервис, поскольку количество камер и автомобилей в городе потенциально может расти, таким образом структура будет примерно следующей: запросы идут на балансировщик нагрузки, который будет распределять поступающие запросы между одинаковыми микросервисами с ML-моделью, в случае если они не будут справляться с расчетами мы просто дублируем блок с ML-моделью и подключаем к балансировщику, таким образом сами микросервисы не являются точкой отказа, если один из них упадет то расчеты продолжатся

Не думаю что есть необходимость усложнять структуру микросервиса, нам потребуется всего один http запрос.
Можно тащить все 10 фич в заголовке запроса get или же отправлять json файл телом запроса put, вопрос будет состоять в том 
в каком виде нам удобнее отправлять данные на наш сервис.
Можно сказать что первый подход выглядит более каноничным, но второй более здравым, поскольку в данных которые мы передаем запросом есть структуры типа списков.
Однако, я все же буду пользоваться запросом get

Сервис пишем на flask

In [ ]:
# Код для наглядности

from flask import Flask, jsonify, abort, make_response, request
from ml.pick_regno import pick_regno
import logging

model_path = 'ml/models/micromodel.cbm'

logging.basicConfig(filename='logs/logs.log', level=logging.DEBUG)

app = Flask(__name__)


def launch_task(regno_recognize,
                afts_regno_ai,
                recognition_accuracy,
                afts_regno_ai_score,
                afts_regno_ai_char_scores,
                afts_regno_ai_length_scores,
                camera_type,
                camera_class,
                time_check,
                direction,
                api,
                ):

    logging.info('making prediction...')
    prediction = pick_regno(regno_recognize,
                            afts_regno_ai,
                            recognition_accuracy,
                            afts_regno_ai_score,
                            afts_regno_ai_char_scores,
                            afts_regno_ai_length_scores,
                            camera_type,
                            camera_class,
                            time_check,
                            direction,
                            model_name=model_path
                            )

    if api == 'v1.0':
        res_dict = {'result': prediction.tolist()}

        logging.info('getting response')
        return res_dict
    else:
        res_dict = {'error': 'API doesnt exist'}

        logging.warning("api doesn't exist")
        return res_dict


@app.errorhandler(404)
def not_found(error):
    logging.warning('page not found')
    return make_response(jsonify({'code': 'PAGE_NOT_FOUND'}), 404)


@app.errorhandler(500)
def server_error(error):
    logging.warning('server error')
    return make_response(jsonify({'code': 'INTERNAL_SERVER_ERROR'}), 500)


@app.route('/regno/api/v1.0/getpred', methods=['GET'])
def get_task():
    logging.info('launching task')
    result = launch_task(request.args.get('regno_recognize'),
                         request.args.get('afts_regno_ai'),
                         request.args.get('recognition_accuracy'),
                         request.args.get('afts_regno_ai_score'),
                         request.args.get('afts_regno_ai_char_scores'),
                         request.args.get('afts_regno_ai_length_scores'),
                         request.args.get('camera_type'),
                         request.args.get('camera_class'),
                         request.args.get('time_check'),
                         request.args.get('direction'),
                         'v1.0')

    return make_response(jsonify(result), 200)

Функция get_task получает фичи из запроса и запускает выполнение функции launch_task, которая уже работает со скриптом запуска модели и предоставленной функцией pick_regno.

В текущем формате сервис имеет минимальное логирование и может быть запущен на локальной машине со всем зависимостями.
Однако, если нагрузка будет слишком большой, то сервис может не справится и возникнут проблемы, для этого будет необходимо масштабировать систему и отправлять запросы на несколько микросервисов, поэтому нужен балансировщик нагрузки.

Однако, прежде чем работать с балансировщиком засунем микросервис в докер контейнер. 

Содержимое Dockerfile для наглядности:

In [ ]:
FROM python:3.9

RUN python -m pip install flask flask-cors gunicorn numpy pandas catboost

WORKDIR /ml_service

ADD app app
ADD logs logs
ADD ml ml
ADD requirements.txt requirements.txt

EXPOSE 5000

CMD [ "gunicorn", "--bind", "0.0.0.0:5000", "app.server:app" ]

Далее воспользуемся балансировкой нагрузки nginx, по умолчанию будем пользоваться балансировкой round robin, поскольку предполагаем, что сложность распределяемых запросов одинакова (мы всегда передаем 10 фич в модель) и серверы одинаково производительны

Конфиг для nginx балансера nginx.conf:

In [ ]:
user  nginx;events {
    worker_connections   1000;
}
http {
        server {
              listen 4000;
              location / {
                proxy_pass http://regno-microservice:5000;
              }
        }
}

Далее с помощью docker-compose соберем несколько контейнеров с нашими микро-сервисами и балансировщик в одну виртуальную сеть

конфиг docker-compose.yaml:

In [ ]:
version: '3.7'

services:
    regno-microservice:
        build:
            context: ./regno-microservice
        image: ml_service_test
        restart: unless-stopped
        expose:
            - "5000"
        deploy:
            replicas: 3 # Количество реплик микросервиса можно изменить на необходимое


    nginx-balancer:
        image: nginx
        container_name: nginx-balancer
        restart: unless-stopped
        volumes:
            - ./nginx-balancer/nginx.conf:/etc/nginx/nginx.conf:ro
        depends_on:
            - regno-microservice
        ports:
            - "5000:4000"

Все что необходимо собрано, конфиги прописаны, пора проверить работоспособность

Для проверки работоспособности необходим запущенный докер

In [ ]:
# Собираем образ нашего микросервиса консольной командой в корневой директории проекта:
    
docker build -t ml_service_test ./regno-microservice

# Для проверки работоспособности одного микросервиса без балансировщика можно запустить один контейнер консольной командой: 

docker run -d -p 5000:5000 ml_service_test

# Для проверки работоспособности сервиса полностью в корневой папке проекта запускаем консольную команду:

docker-compose up --build

После запуска контейнеров можно отправлять запросы на http://localhost:5000

Поскольку микросервис получает запрос в заголовке, то структура запроса выглядит следующим образом:

In [51]:
f'http://localhost:{port}/regno/api/v1.0/getpred?\
regno_recognize={regno_recognize}\
&afts_regno_ai={afts_regno_ai}\
&recognition_accuracy={recognition_accuracy}\
&afts_regno_ai_score={afts_regno_ai_score}\
&afts_regno_ai_char_scores={afts_regno_ai_char_scores}\
&afts_regno_ai_length_scores={afts_regno_ai_length_scores}\
&camera_type={camera_type}\
&camera_class={camera_class}\
&time_check={time_check}\
&direction={direction}'

'http://localhost:5000/regno/api/v1.0/getpred?regno_recognize=О041ВВ797&afts_regno_ai=О041ВВ777&recognition_accuracy=2.25&afts_regno_ai_score=0.9240373969078064&afts_regno_ai_char_scores=[0.9724164605140686, 0.999618649482727, 0.9999938011169434, 0.9997884631156921, 0.9998886585235596, 0.9996151924133301, 0.9996050000190735, 0.9517149925231934, 0.9999496936798096]&afts_regno_ai_length_scores=[1.2026991624836114e-09, 1.2061864840262615e-09, 1.1980743064299304e-09, 1.2100438429030191e-09, 1.20562060335061e-09, 1.2088351430961097e-09, 1.2120812131755088e-09, 1.0469466182883025e-08, 1.1695751709339675e-06, 0.9999988079071045, 1.2070656696394622e-09]&camera_type=Стационарная&camera_class=Астра-Трафик&time_check=2021-08-07 08:10:42&direction=1'

Зададим параметры для запроса

In [52]:
features = list(data.iloc[0])

regno_recognize, afts_regno_ai, recognition_accuracy, afts_regno_ai_score, afts_regno_ai_char_scores, afts_regno_ai_length_scores, camera_type, camera_class, time_check, direction = features

recognition_accuracy = str(recognition_accuracy)
afts_regno_ai_score = str(afts_regno_ai_score)
direction = str(direction)

In [53]:
import requests

In [54]:
port = 5000

r = requests.get(f'http://localhost:{port}/regno/api/v1.0/getpred?\
regno_recognize={regno_recognize}\
&afts_regno_ai={afts_regno_ai}\
&recognition_accuracy={recognition_accuracy}\
&afts_regno_ai_score={afts_regno_ai_score}\
&afts_regno_ai_char_scores={afts_regno_ai_char_scores}\
&afts_regno_ai_length_scores={afts_regno_ai_length_scores}\
&camera_type={camera_type}\
&camera_class={camera_class}\
&time_check={time_check}\
&direction={direction}')
print('Ответ на запрос:', r.content)

Ответ на запрос: b'{"result":[0.9833415173975795,0.01665848260242052]}\n'


Сервис отвечает на запрос

Далее эту структуру запроса можно использовать для проверки нагрузки на сервис

In [57]:
response_lst = []

start_time = time.time()

for i in range(100):
    features = list(data.iloc[i])
    regno_recognize, afts_regno_ai, recognition_accuracy, afts_regno_ai_score, afts_regno_ai_char_scores, afts_regno_ai_length_scores, camera_type, camera_class, time_check,  direction = features
    
    recognition_accuracy = str(recognition_accuracy)
    afts_regno_ai_score	 = str(afts_regno_ai_score)
    direction = str(direction)
    
    r = requests.get(f'http://localhost:5000/regno/api/v1.0/getpred?\
regno_recognize={regno_recognize}\
&afts_regno_ai={afts_regno_ai}\
&recognition_accuracy={recognition_accuracy}\
&afts_regno_ai_score={afts_regno_ai_score}\
&afts_regno_ai_char_scores={afts_regno_ai_char_scores}\
&afts_regno_ai_length_scores={afts_regno_ai_length_scores}\
&camera_type={camera_type}\
&camera_class={camera_class}\
&time_check={time_check}\
&direction={direction}')
    response_lst.append(r.content)
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.110208034515381 seconds ---


In [58]:
response_lst

[b'{"result":[0.9833415173975795,0.01665848260242052]}\n',
 b'{"result":[0.9958559500374642,0.004144049962535818]}\n',
 b'{"result":[0.20706936700859901,0.792930632991401]}\n',
 b'{"result":[0.9547414527375759,0.04525854726242409]}\n',
 b'{"result":[0.29239232580939667,0.7076076741906033]}\n',
 b'{"result":[0.43860728748476674,0.5613927125152333]}\n',
 b'{"result":[0.8861429184180072,0.11385708158199276]}\n',
 b'{"result":[0.16968317013792045,0.8303168298620796]}\n',
 b'{"result":[0.006357257940204031,0.993642742059796]}\n',
 b'{"result":[0.09151942110388689,0.9084805788961131]}\n',
 b'{"result":[0.0033234264663093427,0.9966765735336907]}\n',
 b'{"result":[0.01324052745757831,0.9867594725424217]}\n',
 b'{"result":[0.29978539896753675,0.7002146010324632]}\n',
 b'{"result":[0.6369831257645788,0.3630168742354212]}\n',
 b'{"result":[0.9774741131728542,0.0225258868271458]}\n',
 b'{"result":[0.0239718612762585,0.9760281387237415]}\n',
 b'{"result":[0.9930100837540468,0.006989916245953274]}\n

По всем 100 запросам получили ответ за 2 секунды на персональном компьютере с Intel(R) Core(TM) i3-7100 CPU @ 3.90GHz. При выполнении в рамках серверной среды, мы можешь масштабировать наши микросервисы и использовать нормальное серверное железо для обработки запросов